<h1 style="background-color:rgb(67, 77, 86);
           font-size:300%;
           font-style: oblique;
           color:white;
           text-align:center;
           margin: auto;
           padding: 20px;">Predicting Bank Churners</h1>

<a id="1.2"></a>
<h2 style="background-color:rgb(141, 153, 165);
           font-size:250%;
           color:white;
           text-align:center;
           margin: auto;
           padding: 10px;">Chapter 5. Spot Check Version 1</h2>

<a id='1.1'>
    <h2 style='font-size:180%;'>
        Mission</h2></a>

<figure>
    <blockquote cite='https://www.kaggle.com/sakshigoyal7/credit-card-customers/tasks?taskId=2729'>
        <p style='font-size:110%;
                  color:hsl(208, 12%, 30%);'><i>Our top priority in this business problem is to identify customers who are getting churned. Even if we predict non-churning customers as churned, it won't harm our business. But predicting churning customers as non-churning will do. So recall needs to be higher. Till now, I have managed to get a recall of 62%.</i></p>
    </blockquote>
    <figcaption>—Sakshi Goyal, <cite>Credit Card Customers, Kaggle</cite></figcaption>

<a id='4.1'>
    <h2 style='font-size:180%;'>
        Libraries</h2></a>

In [2]:
# binary classification spot check script
import time
import warnings
import pandas as pd
import numpy as np
from matplotlib import pyplot

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.combine import SMOTETomek
from imblearn.combine import SMOTEENN

In [3]:
# settings
%matplotlib inline
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.options.display.float_format = '{:,.2f}'.format
np.set_printoptions(suppress=True, precision=3)

In [4]:
%%html
<style>
/* CSS styles for pandas dataframe */
.dataframe th {
    font-size: 16px;
}
.dataframe td {
    font-size: 14px;
}
</style>

In [5]:
time_0 = time.perf_counter()

<a id="1.2"></a>
<h2 style="background-color:rgb(141, 153, 165);
           font-size:250%;
           color:white;
           text-align:center;
           margin: auto;
           padding: 10px;">Spot Check for Model & Scaler</h2>

## Load Data

## Evaluate Models

### Performance Metrics

In [ ]:
def perf_metrics(y_test, y_pred):
    dic = {}
    dic['accuracy'] = round(accuracy_score(y_test, y_pred), 2)
    dic['precision'] = round(precision_score(y_test, y_pred), 2)
    dic['recall'] = round(recall_score(y_test, y_pred), 2)
    dic['f1'] = round(f1_score(y_test, y_pred), 2)
    dic['f2'] = round(fbeta_score(y_test, y_pred, beta=2), 2)
    return dic

### Result Summary

In [ ]:
def result_rskf(x, y, pipeline, mod_disp_name, n_splits=5, n_repeats=3):
   
    # define cv method
    cv = RepeatedStratifiedKFold(
        n_splits=n_splits, n_repeats=n_repeats, random_state=1)  
    
    # define performance metrics
    scoring = {
        'accuracy':'accuracy', 'precision':'precision', 'recall':'recall', 'f1':'f1', 
        'f2':make_scorer(fbeta_score, beta=2)} # dict val = scorer fct or predefined metric str  
    
    # evaluate result
    result = cross_validate(
        pipeline, x, y, cv=cv, 
        scoring=scoring, return_train_score=True, n_jobs=-1)
        
    # make a summary table
    df = pd.DataFrame(
        (k, mean(v), std(v)) for k,v in result.items()
        ).rename({0:'metric', 1:'mean', 2:'std'}, axis=1
                ).set_index('metric')
    df.index.name = None
    df.columns = pd.MultiIndex.from_product([[mod_disp_name],df.columns])
    
    return df, result

In [ ]:
def result_tts(x, y, pipeline, mod_disp_name):
    
    # define cv method
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.2, random_state=1, shuffle=True, stratify=y)
    
    # evaluate result   
    time_0 = time.time()
    pipeline.fit(x_train, y_train)
    time_1 = time.time()
    y_pred = pipeline.predict(x_test)
    time_2 = time.time()
    result = {}
    result['fit_time'] = round(time_1-time_0, 2)
    result['score_time'] = round(time_2-time_1, 2)
    result['accuracy'] = round(accuracy_score(y_test, y_pred), 2)
    result['precision'] = round(precision_score(y_test, y_pred), 2)
    result['recall'] = round(recall_score(y_test, y_pred), 2)
    result['f1'] = round(f1_score(y_test, y_pred), 2)
    result['f2'] = round(fbeta_score(y_test, y_pred, beta=2), 2)
    conf_mat = confusion_matrix(y_test, y_pred, labels=[1,0])
    
    # make a summary table    
    df = pd.DataFrame(result, index=[mod_disp_name]).T
    
    return df, result, conf_mat, y_pred, y_test

In [ ]:
def summary_by_mod(x, y, models, scalers, result_func=result_rskf, **n_splits_and_repeats):
    results = []
    time_0 = time.time() # for all methods in pipeline
    for scaler in scalers:
        results_models = []
        time_1 = time.time() # for each scaler
        print(f'Scaler: {scaler[0]}\n')
        for model in models:
            time_2 = time.time() # for each model
            pipeline = Pipeline([('s', scaler[1]), ('m', model[1])])
            if result_func==result_rskf:
                n_splits, n_repeats = (i for i in n_splits_and_repeats.values())
                results_model = result_func(x, y, pipeline, model[0], n_splits, n_repeats)[0]
            else:
                results_model = result_func(x, y, pipeline, model[0])[0]
            print(f'Model {model[0]} Runtime: {time.strftime("%M:%S", time.gmtime(time.time()-time_2))}')
            results_models.append(results_model)
        print(f'Scaler {scaler[0]} Avg Runtime per Model: {time.strftime("%M:%S", time.gmtime((time.time()-time_1)/len(models)))}\n\n')
        results.append(results_models)
    print(f'Total Runtime: {time.strftime("%M:%S", time.gmtime(time.time()-time_0))} min')
    return results

### Models

In [ ]:
# evaluate a single model
def evaluate_model(X, y, model, resampler, folds, metric, pipe_func):
    
    # create the pipeline
    pipeline = pipe_func(model, resampler)
    
    # define cv method
    cv = RepeatedStratifiedKFold(
        n_splits=10, n_repeats=5, random_state=5)  
    
    # define performance metrics
    scoring = {
        'accuracy':'accuracy', 'precision':'precision', 'recall':'recall', 'f1':'f1', 
        'f2':make_scorer(fbeta_score, beta=2)} # dict val = scorer fct or predefined metric str  
    
    # evaluate result
    scores = cross_validate(
        pipeline, x, y, cv=cv, 
        scoring=scoring, return_train_score=True, n_jobs=-1)
    
    return scores


# evaluate a model and try to trap errors and and hide warnings
def robust_evaluate_model(X, y, model, resampler, folds, metric, pipe_func):
    scores = None
    try:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            scores = evaluate_model(X, y, model, resampler, folds, metric, pipe_func)
    except:
        scores = None
    return scores


# evaluate a dict of models {name:object}, returns {name:score}
def evaluate_models(X, y, models, resamplers, pipe_funcs, folds=10, metric='recall'):
    results = dict()
    for name_model, model in models:
        for name_resampler, resampler in resamplers:
            # evaluate model under each preparation function
            for i in range(len(pipe_funcs)):
                # evaluate the model
                scores = robust_evaluate_model(X, y, model, resampler, folds, metric, pipe_funcs[i])
                # update name
                run_name = str(i) + '_' + name_model + '_' + name_resampler
                # show process
                if scores is not None:
                    # store a result
                    results[run_name] = scores
                    mean_score, std_score = np.nanmean(scores), np.nanstd(scores)
                    print('>%s: %.3f (+/-%.3f)' % (run_name, mean_score, std_score))
                else:
                    print('>%s: error' % run_name)
    return results

## Examine Results

In [ ]:
# print and plot the top n results
def summarize_results(results, maximize=True, top_n=10):
	# check for no results
	if len(results) == 0:
		print('no results')
		return
	# determine how many results to summarize
	n = min(top_n, len(results))
	# create a list of (name, mean(scores)) tuples
	mean_scores = [(k, np.nanmean(v)) for k,v in results.items()]
	# sort tuples by mean score
	mean_scores = sorted(mean_scores, key=lambda x: x[1])
	# reverse for descending order (e.g. for accuracy)
	if maximize:
		mean_scores = list(reversed(mean_scores))
	# retrieve the top n for summarization
	names = [x[0] for x in mean_scores[:n]]
	scores = [results[x[0]] for x in mean_scores[:n]]
	# print the top n
	print()
	for i in range(n):
		name = names[i]
		mean_score, std_score = np.nanmean(results[name]), np.nanstd(results[name])
		print('Rank=%d, Name=%s, Score=%.3f (+/- %.3f)' % (i+1, name, mean_score, std_score))
	# boxplot for the top n
	pyplot.boxplot(scores, labels=names)
	_, labels = pyplot.xticks()
	pyplot.setp(labels, rotation=90)
	pyplot.savefig('spotcheck.png')

In [6]:
# load the dataset, returns X and y elements
def load_dataset():
    d = pd.read_csv('source/d_num.csv')
    d_values = d.values
    x, y = d_values[:,1:], d_values[:,:1].ravel()
    return x, y

In [9]:
# models
logit = LogisticRegression(random_state=5)
ada = AdaBoostClassifier(random_state=5)
gb = GradientBoostingClassifier(random_state=5)

# scalers
rs = ('RS', RobustScaler())
qs = ('QT', QuantileTransformer())

def pipeline_scaler_ENN(scaler, model):
    steps = list()
    # normalization
    steps.append((scaler[0], scaler[1]))
    # standardization
    steps.append(('SS', StandardScaler()))
    # the resampler
    steps.append(('RSP', SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='majority'))))
    # the model
    steps.append(('MOD', model))
    # create pipeline
    pipeline = Pipeline(steps=steps)
    return pipeline

In [11]:
pipeline_scaler_ENN(qs, gb)

Pipeline(steps=[('QT', QuantileTransformer()), ('SS', StandardScaler()),
                ('RSP',
                 SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='majority'))),
                ('MOD', GradientBoostingClassifier(random_state=5))])

In [ ]:
# d = pd.read_csv('source/d_num.csv')
# d_values = d.values
# x, y = d_values[:,1:], d_values[:,:1].ravel()

In [15]:
# d = pd.read_csv('source/d_num.csv')
# X, y = d.iloc[:,1:], d.iloc[:,:1]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.
pca = PCA()
# set the tolerance to a large value to make the example faster
logistic = LogisticRegression(max_iter=10000, tol=0.1)
pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

X_digits, y_digits = datasets.load_digits(return_X_y=True)

# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    'pca__n_components': [5, 15, 30, 45, 64],
    'logistic__C': np.logspace(-4, 4, 4),
}
search = GridSearchCV(pipe, param_grid, n_jobs=-1)
search.fit(X_digits, y_digits)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

# Plot the PCA spectrum
pca.fit(X_digits)

fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(6, 6))
ax0.plot(np.arange(1, pca.n_components_ + 1),
         pca.explained_variance_ratio_, '+', linewidth=2)
ax0.set_ylabel('PCA explained variance ratio')

ax0.axvline(search.best_estimator_.named_steps['pca'].n_components,
            linestyle=':', label='n_components chosen')
ax0.legend(prop=dict(size=12))

In [ ]:
# %% Use RandomSearchCV to tune parameters for RFC 

# Takes a while to compute and uses a lot of CPU 
# https://stats.stackexchange.com/questions/186182/a-way-to-maintain-classifiers-recall-while-improving-precision
#model
# https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

print('Running RandomizedSearchCV')
# default RFClassifier but set random_state = 0 to keep consistent results
MOD = RandomForestClassifier(random_state=0) 
#Implement RandomSearchCV
# Number of trees in random forest [100,150,...,500]
n_estimators = [int(x) for x in np.arange(start = 100, stop = 501, step = 50)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.arange(start = 20, stop = 101, step = 20)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
scoreFunction = {"recall": "recall"}
# run a RandomizedSearchCV with 3 folds and 25 iterations 
random_search = RandomizedSearchCV(MOD,
                                   param_distributions = random_grid,
                                   n_iter = 25,
                                   scoring = scoreFunction,               
                                   refit = "recall",
                                   return_train_score = False,
                                   random_state = 0,
                                   verbose = 2,
                                   cv = 3,
                                   n_jobs = -1) 

#trains and optimizes the model
random_search.fit(X_train80, y_train80)

print('Finished RandomizedSearchCV ')

In [ ]:
# example of grid searching key hyperparameters for GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
# define dataset
X, y = load_dataset()
# define models and parameters
model = pipeline_scaler_ENN(qs, gb)
n_estimators = [100, 300, 800]
learning_rate = [0.001, 0.01, 0.1]
subsample = [0.5, 0.7, 1.0]
max_depth = [3, 7, 9]
# define grid search
grid = dict(learning_rate=learning_rate, n_estimators=n_estimators, subsample=subsample, max_depth=max_depth)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='recall', error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

## Run Results

Across all models, the SMOTE oversampling method in conjunction with the Edited Nearest Neighbor undersampling performed the best. Among the predictive algorithms, GB, ADA, and LR look promising. In the next session, we will narrow down our models and jump into hyperparameter tuning.

In [ ]:
# load dataset
X, y = load_dataset()
# get model list
models = define_models()
# get model list
resamplers = define_resamplers()
# define transform pipelines
pipelines = [pipeline_QTSS_ENN, pipeline_RSSS_ENN]
# evaluate models
results = evaluate_models(X, y, models, resamplers, pipelines)
# summarize results
summarize_results(results)

## Calculate Runtime

In [ ]:
time_1 = time.perf_counter()

In [ ]:
print(f'Finished in {round(time_1-time_0, 2):,} second(s) or {round((time_1-time_0)/60, 2)} minute(s).')

In [ ]:
# Rank=1, Name=2_GB_800_SM_ENN, Score=0.914 (+/- 0.063)
# Rank=2, Name=2_ADA_800_SM_ENN, Score=0.910 (+/- 0.056)
# Rank=3, Name=1_GB_800_SM_ENN, Score=0.908 (+/- 0.087)
# Rank=4, Name=1_ADA_800_SM_ENN, Score=0.906 (+/- 0.063)
# Rank=5, Name=1_LR_SM_ENN, Score=0.870 (+/- 0.107)
# Rank=6, Name=2_LR_SM_ENN, Score=0.868 (+/- 0.075)
# Rank=7, Name=2_MLP_1000_SM_ENN, Score=0.867 (+/- 0.078)
# Rank=8, Name=1_GB_800_SM_TM, Score=0.859 (+/- 0.099)
# Rank=9, Name=2_GB_800_SM, Score=0.857 (+/- 0.099)
# Rank=10, Name=1_GB_800_SM, Score=0.857 (+/- 0.096)